## CS431/631 Data Intensive Distributed Analytics
### Winter 2021 - Assignment 2
---

**Please edit this (text) cell to provide your name and UW student ID number!**
* **Name:** Yifu Zhang
* **ID:** y2644zha

---
#### Overview
For this assignment, you will be using Python and Spark to analyze the [pointwise mutual information (PMI)](http://en.wikipedia.org/wiki/Pointwise_mutual_information) of tokens in the text of Shakespeare's plays.    For this assignment, you will need the same text file (`Shakespeare.txt`) and Python tokenizer module, `simple_tokenize.py`, that you used for the first two assignments.    You will also use the same definition of PMI that was used for [Assignment 1](https://www.student.cs.uwaterloo.ca/~cs451/assignment1-431.html).

We first need to install Apache Spark. Run the next block to download and install Spark. It will take about a minute to finish the instalation.

In [1]:
!apt-get update -qq > /dev/null
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz
!tar xf spark-2.4.7-bin-hadoop2.7.tgz
!pip install -q findspark

To use Spark from within a Python program, it is first necessary to tell the Python interpreter where the Spark installation is located. The code in the following cell tells Python how to find this Spark installation. This code creates SparkContext (sc) for you. Do not change this block.

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.7-bin-hadoop2.7"

import findspark
findspark.init()

from pyspark import SparkContext, SparkConf

spark_conf = SparkConf()\
  .setAppName("YourTest")\
  .setMaster("local[*]")

sc = SparkContext.getOrCreate(spark_conf)

Once Python knows where Spark is located, you can create a `SparkContext`.   All Spark commands must run within an active `SparkContext`.   The code below will create a `SparkContext`, and store a reference to the context in the variable `sc`. 
The `appName` parameter assigns a name of your choosing to the Spark jobs that are created in this context - this is useful mostly for debugging.   The `master` parameter indicates that Spark jobs will run in local mode, using two threads.   This means that your Spark jobs are not really running on a cluster, and are instead running in a single process on the local machine.   You program Spark jobs the same way whether they run in local mode or on a cluster - the main difference between local and cluster modes is, of course, performance.

Next, let's test that your `SparkContext` has been set up properly by running some simple test code.   This code uses a single Spark job to estimate the value of Euler's number $e$. One way to calculate $e$ is to use the following serries by Jacob Bernoulli:

$p_n = 1 - \frac{1}{1!} + \frac{1}{2!} - \frac{1}{3!} + \cdots + \frac{(-1)^n}{n!} = \sum_{k = 0}^n \frac{(-1)^k}{k!}$

As n tends to infinity, $p_n$ approaches $1/e$.

In the following code,  `parallelize()` and `map()` are Spark *transformations*, and `reduce()` is a Spark *action*.   Study the code in the cell below, then go ahead and run it.   It should take several seconds, since a Spark job is being created and executed, and should print an estimate of $e$ when it finishes.   


In [ ]:
import math

n= 10000
inverse_e = sc.parallelize(range(0, n)).map(lambda x: ((-1)**x) * (1 / math.factorial(x))).reduce(lambda x,y:x+y)
e = 1 / inverse_e
print("e = ", e)

---
#### Question 1  (4/30 marks):

In the following cell, briefly explain how the example works.   What is the Spark job doing, and how is it used to estimate the value of $e$? How accurate is our estimate?

#### Your answer to Question 1:

1. we first parallelize each number in range(0, $10^4$) into RDD form. Then, we transfer each x into $(-1)^x\frac{1}{x!}$, that is map. Finally, we reduce and sum all of elements togther to get the approximation of e.
2. How? by the Taylor expanision of $e$, we just use MapReduce to implement it.
3. How accurate? As $n$ increases, the accuracy will also increase.

---
### Important

###### The questions that follow ask you to implement functions whose prototypes are given to you. Do **NOT** change the prototypes of the functions. Do **NOT** write code outside of the functions. All necessary code should be included in the function body (except for import statements). You may declare functions inside of the function body. When marking, we will execute your code by calling the functions from an external program, which is why your code cannot rely on statements running outside functions. Please remove any call to the functions that you may have introduced for test purposes before submitting your notebook.

---
#### Question 2  (4/30 marks):

Now it is your turn to write some Spark programs. Start with the simple task of counting the number of *distinct* tokens which appear in `Shakespeare.txt`.   You have already written Python code to do this in Assignment 1, but for this assignment we want you to use Spark to solve the same problem.   You should compare the answer you get using Spark with the answer you got from your pure-Python solution from Assignment 1.   Both answers should, of course, be the same.

Run the next block to download the text file (Shakespeare.txt) and the Python tokenizer module (simple_tokenize.py).

In [26]:
!wget -q https://student.cs.uwaterloo.ca/~cs451/content/cs431/Shakespeare.txt
!wget -q https://student.cs.uwaterloo.ca/~cs451/content/cs431/simple_tokenize.py

Your code should use Spark, not the Python driver code, to read `Shakespeare.txt` and do the counting.   The idea is to use Spark to give you a data-parallel alternative to the sequential Python solution you wrote for Assignment 1.

Write your solution for question 2 in the code cell below, by implementing the `count_distinct_tokens` function. It should use the `SparkContext` which was created previously (referenced by the variable `sc`). The function `count_distinct_tokens` must return the number of distinct tokens.

In [25]:
from simple_tokenize import simple_tokenize
import pandas as pd

# Returns the count of distinct tokens in the `Shakespeare.txt` dataset
def count_distinct_tokens():
    # your solution to Question 2 here
    lines = sc.textFile("Shakespeare.txt").cache()
    tokenize_lines = lines.map(lambda x: simple_tokenize(x)).filter(lambda y: y!=[])
    lines_w_unique_token = tokenize_lines.map(lambda x: pd.unique(x).tolist())
    
    token_dic = lines_w_unique_token.flatMap(lambda lst: iter(lst)).map(lambda word: (word,1)).reduceByKey(lambda x,y: x+y)
    return token_dic.count()

---
#### Question 3  (4/30 marks):

Next, write a Spark program that will count the number of distinct token pairs in `Shakespeare.txt`, as you did in Assignment 1. Again, ensure that the answer that you get using Spark matches the answer you got in the first assignment.

Write your solution for question 3 by implementing the `count_distinct_pairs` function in the code cell below.   It should use the `SparkContext` which was created previously (referenced by the variable `sc`). The function `count_distinct_pairs` must return the number of distinct token pairs.

In [24]:
from simple_tokenize import simple_tokenize
from itertools import permutations

# Returns the count of distinct pairs in the `Shakespeare.txt` dataset
def count_distinct_pairs():
    # your solution to Question 3 here
    lines = sc.textFile("Shakespeare.txt").cache()
    tokenize_lines = lines.map(lambda x: simple_tokenize(x)).filter(lambda y: y!=[])
    lines_w_unique_token = tokenize_lines.map(lambda x: pd.unique(x).tolist())
    
    pairs = lines_w_unique_token.flatMap(lambda line: permutations(line, 2)).filter(lambda keypart: keypart[0]!= keypart[1])
    pair_dic = pairs.map(lambda pair: (pair, 1)).reduceByKey(lambda x,y: x+y)
    return pair_dic.count()
      

---
#### Question 4  (6/30 marks):

Next, write Spark code that will calculate the probability $p(x)$ (as defined in Assignment 1) for every distinct token $x$ in `Shakespeare.txt`.   Your function should return the 50 highest-probability tokens, and their counts and probabilities.

Make sure that your solution calculates probabilities and identifies the 50 highest-probability tokens in a data-parallel fashion, using Spark transformations and actions.   Only the 50 highest-probability tokens (and their counts and probabilities) should be returned by Spark to your driver code.

Write your solution for question 4 by implementing the `top_50_tokens_probabilities` function in the code cell below. It should use the `SparkContext` which was created previously (referenced by the variable `sc`). The function `top_50_tokens_probabilities` must return a list of 50 (probability, count, token) tuples, ordered by probability, that is, the list returned by the function should be of the form: `[(proba1, count1, token1), (proba2, count2, token2), ..., (proba50, count50, token50)]`

In [22]:
from simple_tokenize import simple_tokenize
import pandas as pd


# Returns a list of the top 50 (probability, count, token) tuples, ordered by probability
def top_50_tokens_probabilities():
    # your solution to Question 4 here
    lines = sc.textFile("Shakespeare.txt").cache()
    tokenize_lines = lines.map(lambda x: simple_tokenize(x)).filter(lambda y: y!=[])
    lines_w_unique_token = tokenize_lines.map(lambda x: pd.unique(x).tolist())

    # calculate the number of lines in text
    number_lines = lines.count()

    transit = lines_w_unique_token.flatMap(lambda line: iter(line)).map(lambda word: (word, 1))
    sum_count = transit.reduceByKey(lambda x,y: x+y).map(lambda x: (x[1], x[0])).sortByKey(False)
    ans = sum_count.map(lambda pair: (pair[0]/number_lines, pair[0], pair[1]))
    return ans.take(50)


---
#### Question 5  (6/30 marks):

Next, write the code for the function `PMI` that will take a positive integer threshold $T$ as an argument, and then return all token pairs that co-occur at least $T$ times in `Shakespeare.txt`.   For each such pair $(x,y)$, the function should also return PMI$(x,y)$, the co-occurrence count of the pair, the number of times $x$ appears, and the number of times $y$ appears. You can compare the results produced by this code with the results of Two-Token queries (from Assignment 1) for consistency.

Write your solution for question 5 by implementing the function `PMI` in the code cell below. It should use the `SparkContext` which was created previously (referenced by the variable `sc`). The function `PMI` should return a list of ((token1, token2), pmi, co-occurrence_count, token1_count, token2_count) tuples, that is, the list returned by the function should be of the form: `[((token1, token2), pmi, cooc_count, token1_count, token2_count), (...), ((other_token1, other_token2), other_pmi, other_cooc_count, other_token1_count, other_token2_count)]`.


As always, calculations should be done in data-parallel fashion by using Spark.

In [21]:
from simple_tokenize import simple_tokenize
from math import log
from itertools import permutations

# Returns a list of tuples with the following format:
# ((token1, token2), pmi, co-occurrence_count, token1_count, token2_count)
def PMI(threshold):
    # your solution to Question 5 here
    lines = sc.textFile("Shakespeare.txt").cache()
    tokenize_lines = lines.map(lambda x: simple_tokenize(x)).filter(lambda y: y!=[])
    lines_w_unique_token = tokenize_lines.map(lambda x: pd.unique(x).tolist())

    # number of token pairs
    pairs = lines_w_unique_token.flatMap(lambda line: permutations(line, 2)).filter(lambda keypart: keypart[0]!= keypart[1])
    dic = pairs.map(lambda pair: (pair, 1))
    pair_counts = dic.reduceByKey(lambda x,y: x+y).filter(lambda x: x[1] >= threshold)

    # token counts
    transit = lines_w_unique_token.flatMap(lambda line: iter(line)).map(lambda word: (word, 1))
    sum_count = transit.reduceByKey(lambda x,y: x+y).filter(lambda x: x[1] >= threshold)

    line_count = lines.count() 
    # ans by spark
    trans_1 = pair_counts.map(lambda x: (x[0][0],(x[0][1], x[1]))).join(sum_count)
    trans_2 = trans_1.map(lambda x: (x[1][0][0], (x[0],x[1][0][1],x[1][1]))).join(sum_count)
    trans_3 = trans_2.map(lambda x: ((x[1][0][0],x[0]),x[1][0][1],x[1][0][2],x[1][1]))
    ans = trans_3.map(lambda x: (x[0],log((x[1]/(x[2] * x[3]))*line_count, 10),x[1],x[2],x[3]))
    return ans.collect()


---
#### Question 6  (6/30 marks):

Finally, write Spark code for the function `PMI_one_token`, that will take a positive integer threshold $T$ and a sample size $N$ as arguments. For every token $x$ in `Shakespeare.txt`, your code should find all tokens $y$ that co-occur with $x$ at least $T$ times, as well as PMI$(x,y)$ for each such pair, the co-occurrence count of the pair, the number of times $x$ appears, and the number of times $y$ appears.

For each $x$, the output of your program should be equivalent to the output that would be produced by a One-Token query on $x$ (see Assignment 1), with threshold $T$. Rather than producing output for all possible tokens $x$, the function should produce output only for $N$ different $x$'s, chosen at random from among all distinct tokens in the input file.

Write your solution for question 6 by implementing the function `PMI_one_token` in the code cell below. It should use the `SparkContext` which was created previously (referenced by the variable `sc`). The function `PMI_one_token` should return a list of $N$ tuples of the form `(token, [ list_of_cooccurring_tokens ])`, where each entry in `list_of_cooccurring_tokens` is of the form `((token1, token2), pmi, cooc_count, token1_count, token2_count)`.

For instance, if $N$ = 2 and the randomly selected tokens are "if" and "you", a valid format for the value returned by `PMI_one_token` would be:
```
[('if', [(('if', 'you'), -0.09813466615111513, 1975, 16624, 18237), (('if', 'a'), 0.03856379243802408, 1568, 16624, 10569)]), ('you', [(('you', 'if'), -0.09813466615111513, 1975, 18237, 16624)])]
```

Hint: Sampling must be done at the very last step. Hint: there is an action that returns a sample subset from an RDD.


In [20]:
from simple_tokenize import simple_tokenize
from math import log

# Returns a list of samp_size tuples with the following format:
# (token, [ list_of_cooccurring_tokens ])
# where list_of_cooccurring_tokens is of the form
# [((token1, token2), pmi, cooc_count, token1_count, token2_count), ...]
def PMI_one_token(threshold, samp_size):
    # your solution to Question 6 here

    lines = sc.textFile("Shakespeare.txt").cache()
    tokenize_lines = lines.map(lambda x: simple_tokenize(x)).filter(lambda y: y!=[])
    lines_w_unique_token = tokenize_lines.map(lambda x: pd.unique(x).tolist())

    pairs = lines_w_unique_token.flatMap(lambda line: permutations(line, 2)).filter(lambda keypart: keypart[0]!= keypart[1])
    dic = pairs.map(lambda pair: (pair, 1))

    pair_dic = dic.reduceByKey(lambda x,y: x+y).filter(lambda x: x[1]>threshold)
    token_dic = lines_w_unique_token.flatMap(lambda lst: iter(lst)).map(lambda word: (word,1)).reduceByKey(lambda x,y: x+y)

    line_count = lines.count()

    trans_1 = pair_dic.map(lambda x: (x[0][0],(x[0][1],x[1]))).join(token_dic)
    trans_2 = trans_1.map(lambda x: (x[1][0][0],(x[0],x[1][0][1],x[1][1]))).join(token_dic)
    trans_3 = trans_2.map(lambda x: ((x[1][0][0],x[0]),x[1][0][1],x[1][0][2],x[1][1]))
    trans_4 = trans_3.map(lambda x: (x[0][0], [(x[0],log(x[1]/x[2]/x[3]*line_count, 10),x[1],x[2],x[3])]))
    ans = trans_4.reduceByKey(lambda x,y: x+y)

    # take sample
    return ans.takeSample(False, samp_size)

---
Don't forget to save your workbook!   When you are finished and you are ready to submit your assignment, download your notebook file (.ipynb) from the hub to your machine, and then follow the submission instructions in the assignment.